## Name : Adwaiy Singh
## Reg. No. : 220968424
## Sec. : A
## Batch: 2
## WEEK 5

### MULTI-ARMED BANDITS – AD OPTIMIZATION

Consider the dataset **"Ads_Clicks,"** which contains information about user interactions with advertisements over time. An advertising company is running **18 different ads** on a webpage, all targeted toward a similar audience. The dataset records whether a user clicked at a given time step. Each column corresponds to a specific ad, where **YES(1) indicates that the ad was clicked, and NO(0) indicates that it was not.**

##### Consider the attached csv fi each adle.
    1.	Define the multi-armed bandit (MAB) problem in the context of ad optimization, considering how an agent selects among multiple ads to maximize clicks.
    2.	How does the exploration-exploitation trade-off influence decision-making in this scenario?
    3.	Implement the ε-greedy algorithm to optimize ad selection and compute the total rewards after 2000-time steps for:ε = 0.05 and ε = 0.2
    4.	Compare the effect of different ε values on total rewards and action selection.
    5.	Implement the UCB method with an exploration factor c = 2.0 and compute total rewards after 2000-time steps.
    6.	How does increasing or decreasing the exploration factor c affect the performance?
    7.	Analyze how the estimated action values (Q-values) compare to the actual optimal action in both ε-greedy and UCB methods.
    8.	Which approach leads to a better approximation of the optimal action?
    9.	Evaluate how the performance of ε-greedy and UCB changes when the time horizon is extended to 5000-time steps instead of 2000-time steps.
    10.	Does a longer time horizon reduce the impact of exploration parameters (ε or c) on total rewards?


#### Imports

In [24]:
import pandas as pd
import numpy as np
import random
import math

#### Loading data

In [31]:
AdsDf = pd.read_csv("Ad Click Data.csv")

AdsDf.head()

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Ad Topic Line,City,Male,Country,Timestamp,Clicked on Ad
0,68.95,35,61833.90,256.09,Cloned 5thgeneration orchestration,Wrightburgh,0.0,Tunisia,3/27/2016 0:53,0
1,80.23,31,68441.85,193.77,Monitored national standardization,West Jodi,1.0,Nauru,4/4/2016 1:39,0
2,69.47,26,59785.94,236.50,Organic bottom-line service-desk,Davidton,0.0,San Marino,3/13/2016 20:35,0
3,74.15,29,54806.18,245.89,Triple-buffered reciprocal time-frame,West Terrifurt,1.0,Italy,1/10/2016 2:31,0
4,68.37,35,73889.99,225.58,Robust logistical utilization,South Manuel,0.0,Iceland,6/3/2016 3:36,0


In [32]:
clickedAdsDf = pd.read_csv("Clicked Ads Dataset.csv")
clickedAdsDf = clickedAdsDf.iloc[:,1:]

clickedAdsDf.head()

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Male,Timestamp,Clicked on Ad,city,province,category
0,68.95,35,432837300.0,256.09,Perempuan,3/27/2016 0:53,No,Jakarta Timur,Daerah Khusus Ibukota Jakarta,Furniture
1,80.23,31,479092950.0,193.77,Laki-Laki,4/4/2016 1:39,No,Denpasar,Bali,Food
2,69.47,26,418501580.0,236.50,Perempuan,3/13/2016 20:35,No,Surabaya,Jawa Timur,Electronic
3,74.15,29,383643260.0,245.89,Laki-Laki,1/10/2016 2:31,No,Batam,Kepulauan Riau,House
4,68.37,35,517229930.0,225.58,Perempuan,6/3/2016 3:36,No,Medan,Sumatra Utara,Finance


#### Preprocessing data

In [33]:
AdsDf.isnull().sum()

Daily Time Spent on Site    13
Age                          0
Area Income                 13
Daily Internet Usage        11
Ad Topic Line                0
City                         1
Male                         3
Country                      9
Timestamp                    0
Clicked on Ad                0
dtype: int64

In [34]:
clickedAdsDf.isnull().sum()

Daily Time Spent on Site    13
Age                          0
Area Income                 13
Daily Internet Usage        11
Male                         3
Timestamp                    0
Clicked on Ad                0
city                         0
province                     0
category                     0
dtype: int64

In [35]:
AdsDf['Daily Time Spent on Site'] = AdsDf['Daily Time Spent on Site'].fillna(AdsDf['Daily Time Spent on Site'].mean())
AdsDf['Area Income'] = AdsDf['Area Income'].fillna(AdsDf['Area Income'].mean())
AdsDf['Daily Internet Usage'] = AdsDf['Daily Internet Usage'].fillna(AdsDf['Daily Internet Usage'].mean())

AdsDf['City'] = AdsDf['City'].fillna(AdsDf['City'].mode()[0])
AdsDf['Male'] = AdsDf['Male'].fillna(AdsDf['Male'].mode()[0])
AdsDf['Country'] = AdsDf['Country'].fillna(AdsDf['Country'].mode()[0])

In [36]:
clickedAdsDf['Daily Time Spent on Site'] = clickedAdsDf['Daily Time Spent on Site'].fillna(clickedAdsDf['Daily Time Spent on Site'].mean())
clickedAdsDf['Area Income'] = clickedAdsDf['Area Income'].fillna(clickedAdsDf['Area Income'].mean())
clickedAdsDf['Daily Internet Usage'] = clickedAdsDf['Daily Internet Usage'].fillna(clickedAdsDf['Daily Internet Usage'].mean())

clickedAdsDf['Male'] = clickedAdsDf['Male'].fillna(clickedAdsDf['Male'].mode()[0])
clickedAdsDf['Clicked on Ad'] = clickedAdsDf['Clicked on Ad'].map({'No': 0, 'Yes': 1})

In [49]:
aaa = clickedAdsDf.copy(deep=True)
aaa

,Daily Time Spent on Site,Age,Area Income,Daily Internet Usage,Male,Timestamp,Clicked on Ad,city,province,category
0,68.950000,35,432837300.0,256.09,Perempuan,3/27/2016 0:53,0,Jakarta Timur,Daerah Khusus Ibukota Jakarta,Furniture
1,80.230000,31,479092950.0,193.77,Laki-Laki,4/4/2016 1:39,0,Denpasar,Bali,Food
2,69.470000,26,418501580.0,236.50,Perempuan,3/13/2016 20:35,0,Surabaya,Jawa Timur,Electronic
3,74.150000,29,383643260.0,245.89,Laki-Laki,1/10/2016 2:31,0,Batam,Kepulauan Riau,House
4,68.370000,35,517229930.0,225.58,Perempuan,6/3/2016 3:36,0,Medan,Sumatra Utara,Finance
...,...,...,...,...,...,...,...,...,...,...
995,72.970000,30,499691990.0,208.58,Laki-Laki,2/11/2016 21:49,1,Semarang,Jawa Tengah,Finance
996,51.300000,45,474475190.0,134.42,Laki-Laki,4/22/2016 2:07,1,Pakanbaru,Riau,Electronic
997,64.929524,51,296910040.0,120.37,Laki-Laki,2/1/2016 17:24,1,Samarinda,Kalimantan Timur,Travel
998,55.550000,19,293445530.0,187.95,Perempuan,3/24/2016 2:35,0,Tangerang,Banten,Bank


In [51]:
aaa = aaa[["category", "Timestamp", "Clicked on Ad"]]
aaa

,category,Timestamp,Clicked on Ad
0,Furniture,3/27/2016 0:53,0
1,Food,4/4/2016 1:39,0
2,Electronic,3/13/2016 20:35,0
3,House,1/10/2016 2:31,0
4,Finance,6/3/2016 3:36,0
...,...,...,...
995,Finance,2/11/2016 21:49,1
996,Electronic,4/22/2016 2:07,1
997,Travel,2/1/2016 17:24,1
998,Bank,3/24/2016 2:35,0


In [54]:
aaa["Timestamp"] = aaa["Timestamp"].apply(lambda x: x.split()[0])
aaa

,category,Timestamp,Clicked on Ad
0,Furniture,3/27/2016,0
1,Food,4/4/2016,0
2,Electronic,3/13/2016,0
3,House,1/10/2016,0
4,Finance,6/3/2016,0
...,...,...,...
995,Finance,2/11/2016,1
996,Electronic,4/22/2016,1
997,Travel,2/1/2016,1
998,Bank,3/24/2016,0


In [65]:
cats = list(aaa.category.unique())
a = {
    date: [0] * 10
    for date in sorted(aaa.Timestamp.unique())
}
cats, a

(['Furniture',
  'Food',
  'Electronic',
  'House',
  'Finance',
  'Travel',
  'Health',
  'Bank',
  'Fashion',
  'Otomotif'],
 {'1/1/2016': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '1/10/2016': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '1/11/2016': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '1/12/2016': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '1/13/2016': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '1/14/2016': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '1/15/2016': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '1/16/2016': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '1/17/2016': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '1/18/2016': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '1/19/2016': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '1/2/2016': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '1/20/2016': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '1/21/2016': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '1/22/2016': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '1/23/2016': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '1/24/2016': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '1/25/2016': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
  '1/26/2016': [0, 0, 0, 0, 0

In [76]:
for cat, time in aaa[aaa["Clicked on Ad"] == 1][["category", "Timestamp"]].values:
    a[time][cats.index(cat)] = 1

In [82]:
pd.DataFrame(a, index=cats).T

,Furniture,Food,Electronic,House,Finance,Travel,Health,Bank,Fashion,Otomotif
1/1/2016,0,0,0,0,1,0,0,0,0,1
1/10/2016,0,0,0,0,0,0,0,0,1,0
1/11/2016,0,0,0,0,0,1,0,1,0,0
1/12/2016,0,0,1,0,0,1,0,0,0,0
1/13/2016,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
7/5/2016,0,0,0,0,1,0,1,0,0,0
7/6/2016,0,1,0,0,1,1,1,0,1,0
7/7/2016,0,0,0,0,0,1,0,0,1,0
7/8/2016,0,0,0,0,0,0,0,0,1,0


In [86]:
float("inf")/1

inf

In [60]:
aaaa = pd.DataFrame(aaa.sort_values("Timestamp").groupby("Timestamp"))
aaaa

,0,1
0,1/1/2016,category Timestamp Clicked on Ad 758 ...
1,1/10/2016,category Timestamp Clicked on Ad 166 ...
2,1/11/2016,category Timestamp Clicked on Ad 644 Fi...
3,1/12/2016,category Timestamp Clicked on Ad 893 ...
4,1/13/2016,category Timestamp Clicked on Ad 518 O...
...,...,...
198,7/5/2016,category Timestamp Clicked on Ad 383 ...
199,7/6/2016,category Timestamp Clicked on Ad 722 Fin...
200,7/7/2016,category Timestamp Clicked on Ad 713 ...
201,7/8/2016,category Timestamp Clicked on Ad 510 ...


### 1.	Define the multi-armed bandit (MAB) problem in the context of ad optimization, considering how an agent selects among multiple ads to maximize clicks.

#### Ans.

There are **18 different ads** being displayed to users, and each ad can either be clicked or not clicked. The goal is to select the best-performing ads, those that are most likely to generate a click, in order to **maximize the total number of clicks** over time.

Each ad corresponds to an "arm" of the bandit, and the agent must select one ad (arm) at each time step. The agent will receive feedback on whether the selected ad was clicked (yes = 1) or not clicked (no = 0).

##### The agent’s task is to balance between:
    Exploration: Trying out different ads to gather more information about their click-through rates (CTRs) and performance.
    Exploitation: Selecting the ads that have already shown a higher likelihood of getting clicked, thereby maximizing the total number of clicks in the long run.

### 2. How does the exploration-exploitation trade-off influence decision-making in this scenario?

#### Ans.

Exploration involves trying out different ads to gather more data about their click-through rates (CTR). Even if an ad has not performed well in the past, exploring it can provide valuable insights that might lead to discovering a more effective ad in the future. This is important for discovering new ads that might perform better over time.

Exploitation involves focusing on ads that have already been shown to perform well (i.e., those with a higher CTR). By exploiting these ads, the agent maximizes immediate clicks, since it is relying on the ads that have historically yielded the best results.

##### The trade-off arises because:

    If the agent explores too much, it may waste resources on ads that aren't performing well, leading to fewer total clicks.
    If the agent exploits too much, it may miss out on discovering better-performing ads, leading to stagnation in click performance.

### 3.	Implement the ε-greedy algorithm to optimize ad selection and compute the total rewards after 2000-time steps for:ε = 0.05 and ε = 0.2

In [ ]:
def epsilon_greedy(epsilon, time_steps, df):
    Q = [0] * n_ads
    N = [0] * n_ads 
    total_reward = 0

    for t in range(time_steps):
        if random.random() < epsilon:
            ad_index = random.randint(0, n_ads - 1)
            chosen_category = ad_categories[ad_index]
        else:
            maxQ = max(Q)
            ad_indices = [i for i, value in enumerate(Q) if value == maxQ]
            ad_index = random.choice(ad_indices)
            chosen_category = ad_categories[ad_index]

        category_rows = df[df['category'] == chosen_category]
        if not category_rows.empty:
            reward = category_rows['Clicked on Ad'].iloc[t % len(category_rows)]
        else:
            reward = 0 
            print(f"Warning: No data found for category {chosen_category}.  Setting reward to 0.")

        N[ad_index] += 1
        Q[ad_index] += (1 / N[ad_index]) * (reward - Q[ad_index])
        total_reward += reward

    return total_reward, Q, N

In [ ]:
def ucb(c, time_steps, df):
    Q = [0] * n_ads
    N = [0] * n_ads
    total_reward = 0

    for t in range(time_steps):
        ucb_values = [Q[a] + c * math.sqrt(math.log(t + 1) / (N[a] + 1e-6)) for a in range(n_ads)]

        ad_index = ucb_values.index(max(ucb_values))
        ad_indices = [i for i, value in enumerate(ucb_values) if value == ucb_values[ad_index]]
        ad_index = random.choice(ad_indices)
        chosen_category = ad_categories[ad_index]

        category_rows = df[df['category'] == chosen_category]
        if not category_rows.empty:
            reward = category_rows['Clicked on Ad'].iloc[t % len(category_rows)]
        else:
            reward = 0 
            print(f"Warning: No data found for category {chosen_category}. Setting reward to 0.")

        # Update
        N[ad_index] += 1
        Q[ad_index] += (1 / N[ad_index]) * (reward - Q[ad_index])
        total_reward += reward

    return total_reward, Q, N